In [ ]:
# 2021年スマブラー格付けチェックの回答をTwitterAPIで取得し、csv出力と各種分析を行う

In [ ]:
# 匿名化のため出力を削除している

### 準備

In [ ]:
import tweepy
import datetime
from pytz import timezone
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option("display.max_colwidth", 80)

In [ ]:
# yyyyMMddHHmmss形式で現在の日本時間を出力
def get_now():
    now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9))) # 日本時刻
    return now.strftime('%Y%m%d%H%M%S')

In [ ]:
# 申請･取得したキーやトークンを入力する

API_KEY = 'pppppppppppppppppppppppppp'
API_SECRET_KEY = 'kkkkkkkkkkkkkkkkkkkkkkkkkkk'
Access_token = 'mmmmmmmmmmmmmmmmmmmmm'
Access_secret = 'nnnnnnnnnnnnnnnnnnnnnn'

In [ ]:
auth = tweepy.OAuthHandler(API_KEY, API_SECRET_KEY)
auth.set_access_token(Access_token, Access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True) # wait_on_rate_limit=True, API上限到達時に自動で待機する

### データ取得

In [ ]:
# ｢Aの部屋に投票した人｣と判定する条件 (Bの部屋についても同様に定める)
# 2021/12/29 18:00 から 2021/12/30 17:59の間に、#Aの部屋 のハッシュタグを含み #Bの部屋 のハッシュタグを含まないツイートをした

# 以下の条件も検討していたが、引用RTは取得できず、RTユーザーは約100件しか取得できなかった(cf.https://mura-shin.com/python_twitter/)ので断念
# 2. 出題ツイートの引用RTにて、文字列"A"を含み"B"を含まないツイートをしている
# 3. 出題ツイートを通常RTした後の10分以内の直近のツイートにて、文字列"A"を含み"B"を含まないツイートをしている

In [ ]:
# 出題ツイートID
nietono_tw_id = 1476116006825521160

In [ ]:
# 回答ツイートとユーザー情報を取得

room_names = ["A", "B"]
tw_data = defaultdict(list)

for vote_to, unvote_to in zip(room_names, reversed(room_names)):
    for tweet in tweepy.Cursor(api.search_tweets, q=f"#{vote_to}の部屋 AND -#{unvote_to}の部屋", since_id=nietono_tw_id, until='2021-12-30_17:59:59_JST', result_type="recent").items():
        if tweet.text[0:4]=="RT @": # 単純RTや、引用RTのRT(例:1476301813599068161)はtweet.text[0:4]=="RT @"となることを利用して除く
            continue
        else:
            tw_data["vote"].append(vote_to)
            tw_data["tw_time"].append(tweet.created_at.astimezone(timezone('Asia/Tokyo'))) # JSTに修正
            tw_data["user_id"].append(tweet.user.id)
            tw_data["user_name"].append(tweet.user.name)
            tw_data["user_screen_name"].append(tweet.user.screen_name)
            tw_data["user_followers_count"].append(tweet.user.followers_count)
            tw_data["url"].append(f"twitter.com/{tweet.user.screen_name}/status/{tweet.id}")
            tw_data["is_quote"].append(tweet.is_quote_status)

df_vote = pd.DataFrame.from_dict(tw_data)
df_vote.to_csv("df_vote_" + get_now() + ".csv", index=False, encoding="utf-8-sig") # 時刻付きで出力
df_vote

### データ加工･出力

In [ ]:
# 何度か分けて取得したデータをマージ。各ユーザーごとに最後の回答だけ残す

df_vote = pd.DataFrame()
for yyyyMMddHHmmss in [20211230173339, 20211230175952, 20211230181450, 20211230183555]:
    df_vote = pd.concat([df_vote, pd.read_csv(f"df_vote_{yyyyMMddHHmmss}.csv")])

df_vote = (df_vote
           .sort_values(["tw_time"])
           .drop_duplicates("user_id", keep="last")
          )

# メインキャラ別集計のため、取り忘れたユーザープロフを取得
user_prof = []
for user_id in df_vote.user_id:
    try:
        user_prof.append(api.get_user(user_id = user_id).description)
    except:
        user_prof.append("")
df_vote["user_prof"] = user_prof

In [ ]:
# 出力前にデータフレーム修正

# 文字列を与えると、ソニック使用者と推定されるかピクオリ使用者と推定されるかそれ以外かを返す
def sonic_pikmin_flg(prof):
    prof = prof.lower() # 小文字に統一
    if any(x in prof for x in ["ソニック", "sonic"]):
        return "sonic"
    if any(x in prof for x in ["ピクオリ","ピクミン","オリマー","アルフ","pikmin","olimar", "alph"]):
        return "pikmin"
    else:
        return "other"

# ユーザープロフから｢ソニック｣｢ピクオリ｣使用者を抽出する
df_vote["main"] = df_vote["user_prof"].apply(sonic_pikmin_flg)

# カラム名修正
df_vote.columns = [x.replace("user","twitter_user").replace("url","vote_url") for x in df_vote.columns]

# 出題後何時間後のツイートかカラム追加
df_vote["delta_hours"] = (df_vote["tw_time"].apply(pd.to_datetime)-pd.to_datetime("2021-12-29 18:00:00+09:00")).dt.total_seconds().apply(lambda x: int(x/3600))

# 出力
df_vote.to_csv("df_vote.csv", index=False, encoding="utf-8-sig")
df_vote

### 集計、図示

In [ ]:
# 投票数集計
# 出題ツイートのアンケとは異なりBの方が多い模様

df_vote.vote.value_counts()
df_vote.vote.value_counts(normalize=True)

In [ ]:
# 回答ごとの基本統計量
df_vote[['vote', 'tw_time', 'twitter_user_followers_count', 'delta_hours']].groupby("vote").describe().T

#### 使用キャラ別集計

In [ ]:
# 動画に出てくるキャラを使用している人は正解率が高いか？

df_vote[df_vote.main.isin(["sonic", "pikmin"])].vote.value_counts()
df_vote[df_vote.main.isin(["sonic", "pikmin"])].vote.value_counts(normalize=True)

In [ ]:
# ソニック使いに限定

df_vote[df_vote.main == "sonic"].vote.value_counts()
df_vote[df_vote.main == "sonic"].vote.value_counts(normalize=True)

In [ ]:
# ピクオリ使いに限定

df_vote[df_vote.main == "pikmin"].vote.value_counts()
df_vote[df_vote.main == "pikmin"].vote.value_counts(normalize=True)

#### フォロワー数ヒストグラム

In [ ]:
# 集計データフレームとフォロワー数の下限と上限を与えると、A、B投票ごとにヒストグラムを表示する
def followers_hist(df, f_min, f_max):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.hist(df[(df.vote == "A") & (df.user_followers_count >= f_min) & (df.user_followers_count <= f_max)].user_followers_count, bins=30, color="red", alpha=0.7, label="A")
    ax.hist(df[(df.vote == "B") & (df.user_followers_count >= f_min) & (df.user_followers_count <= f_max)].user_followers_count, bins=30, color="blue", alpha=0.5, label="B")
    ax.set_xlabel('user_followers_count')
    ax.set_ylabel('counts')
    ax.legend(loc='upper right')
    plt.savefig(f"followers_hist_{f_min}_{f_max}.png")
    plt.show()

In [ ]:
# フォロワー数ヒストグラム。2000人以下を集計
followers_hist(0, 2000)

In [ ]:
# フォロワー数ヒストグラム。1000人以上2万人以下を集計
followers_hist(1000, 20000)

In [ ]:
# フォロワー数箱ひげ図
sns.boxplot(x=df_vote.vote, y=df_vote.twitter_user_followers_count, palette=['red','dodgerblue'])
plt.ylim(0, 2000)
plt.savefig('boxplot_followers.png')

#### 経過時間別集計

In [ ]:
# 経過時間ごとの回答数

df_per_hour = (df_vote
               .groupby(["delta_hours", "vote"], as_index=False).count()
               [["delta_hours", "vote", "vote_url"]]
               .rename(columns={"vote_url":"n_vote"})
              )
ax = df_per_hour.groupby(["delta_hours", "vote"]).sum().unstack().plot.bar(rot=0, color=["r", "b"])
ax.set_ylabel('n_vote')
ax.figure.set_size_inches((13,6))
plt.savefig("AB_per_hours.png")
plt.show()

In [ ]:
# 経過時間ごとの正解率

df_ratio_per_hour = (df_per_hour
                     .assign(n_vote_per_hour = lambda x: x.groupby("delta_hours")["n_vote"].transform("sum"))
                     .pipe(lambda x: x[x.vote == "A"])
                     .assign(A_ratio = lambda x: x.n_vote / x.n_vote_per_hour)
                    )
ax = df_ratio_per_hour.plot.bar(x='delta_hours', y='A_ratio', color="g", rot=0, legend=False)
ax.set_ylabel('A_ratio')
ax.figure.set_size_inches((13,6))
plt.savefig("A_ratio_per_hours.png")
plt.show()

In [ ]:
# 経過時間と正解率の相関

df_ratio_per_hour.corr()